##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [TensorBoard.dev](https://tensorboard.dev) 시작하기

[TensorBoard.dev](https://tensorboard.dev)는 자신의 ML 실험을 업로드하고 모든 사람과 공유할 수 있는 무료 공개 [TensorBoard](https://tensorflow.org/tensorboard) 서비스입니다.

이 노트북에서는 간단한 모델을 훈련하고 TensorBoard.dev에 로그를 업로드하는 방법을 안내합니다. [미리보기](https://tensorboard.dev/experiment/rldGbR8rRHeCEbkK61SWTQ)

### 설정 및 가져오기

이 노트북은 `2.3.0` 이상의 버전에서만 사용할 수 있는 TensorBoard 기능을 사용합니다.

In [ ]:
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp

### 간단한 모델 훈련 및 TensorBoard 로그 생성하기

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

TensorBoard 로그는 [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) 및 [하이퍼 매개변수 콜백](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)을 Keras의 Model.fit()에 전달하여 훈련 중에 생성됩니다. 그러면 이러한 로그를 TensorBoard.dev에 업로드할 수 있습니다.


In [ ]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})

model.fit(
    x=x_train, 
    y=y_train, 
    epochs=5, 
    validation_data=(x_test, y_test), 
    callbacks=[tensorboard_callback, hparams_callback])

### (Jupyter만 해당) TensorBoard.dev 인증하기

**Colab에서는 이 단계가 필요하지 않습니다.**

이 단계에서는 Jupyter 외부의 셸 콘솔에서 인증을 받아야 합니다. 콘솔에서 다음 명령을 실행합니다.

`tensorboard dev list`

이 흐름의 일부로 인증 코드가 제공됩니다. 이 코드는 서비스 약관에 동의하는 데 필요합니다.

### TensorBoard.dev에 업로드하기

TensorBoard 로그를 업로드하면 모든 사람과 공유할 수 있는 URL이 제공됩니다.

업로드된 TensorBoard는 공개되므로 민감한 데이터를 업로드하지 마세요.

전체 logdir이 업로드되면 업로더가 종료됩니다. 이것은 `--one_shot` 플래그에 의해 지정된 동작입니다.

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Simple experiment with MNIST" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

각 개별 업로드에는 고유한 실험 ID가 있습니다. 즉, 동일한 디렉터리로 새 업로드를 시작하면 새 실험 ID를 얻게 됩니다. 업로드한 모든 실험은 https://tensorboard.dev/experiments/에서 볼 수 있습니다. 또는 다음 명령을 사용하여 터미널에서 실험을 나열할 수 있습니다.

```
tensorboard dev list
```

In [ ]:
!tensorboard dev list

### TensorBoard.dev의 스크린샷

https://tensorboard.dev/experiments/로 이동하면 다음과 같이 표시됩니다.

![screenshot of TensorBoard.dev](images/tbdev_getting_started.png "TensorBoard.dev 스크린 샷")

TensorBoard.dev에서 새로운 실험으로 이동할 때의 모습입니다.

![screenshot of TensorBoard.dev experiment dashboard](https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tensorboard/images/tbdev_getting_started.png?raw=true)

### TensorBoard.dev 실험 삭제하기

업로드한 실험을 제거하려면 `delete` 명령을 사용하고 해당 `experiment_id`를 지정합니다. 위 스크린샷에서 experiment_id는 왼쪽 하단 코너에 나열되어 있습니다(`w1lkBAOrR4eH35Y7Lg1DQQ`).

In [ ]:
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE